In [1]:
import pandas as pd
import numpy as np
import openrouteservice # Biblioteca para ORS API
from tqdm.notebook import tqdm # Barra de progresso para Jupyter/IPython
import time # Para possíveis pauses (menos crítico localmente)
import requests

print("Bibliotecas importadas.")

Bibliotecas importadas.


In [2]:
# Caminho para o arquivo de endereços
ADDRESS_FILE_PATH = './data/rideaddress_v1.csv'

# URL do seu servidor ORS local (ajuste a porta se necessário)
ORS_BASE_URL = 'http://localhost:8080/ors'

# Nome do arquivo CSV de saída
OUTPUT_CSV_FILE = './data/ors_local_results.csv'

In [3]:
print(f"Carregando dados de endereço de: {ADDRESS_FILE_PATH}")
try:
    # Tenta carregar com encoding='utf-8' primeiro
    address_df = pd.read_csv(ADDRESS_FILE_PATH, sep=';', encoding='utf-8', low_memory=False)
except UnicodeDecodeError:
    # Se falhar, tente 'latin-1'
    print("Falha ao carregar com UTF-8, tentando Latin-1...")
    address_df = pd.read_csv(ADDRESS_FILE_PATH, sep=';', encoding='latin-1', low_memory=False)

# Seleciona e renomeia colunas
address_df = address_df[["Lat", "Lng", "RideAddressTypeID", "RideID"]]
address_df.rename(columns={
    "Lat": "lat", "Lng": "lng",
    "RideAddressTypeID": "address_type", "RideID": "ride_id"
}, inplace=True)

# --- Limpeza de Coordenadas ---
def clean_coord(coord):
    if pd.isna(coord): return np.nan
    if isinstance(coord, (int, float)): return float(coord)
    if isinstance(coord, str):
        try:
            return float(coord.replace(',', '.'))
        except (ValueError, AttributeError):
            return np.nan
    return np.nan

print("Limpando coordenadas...")
address_df['lat_clean'] = address_df['lat'].apply(clean_coord)
address_df['lng_clean'] = address_df['lng'].apply(clean_coord)

nan_lat_count = address_df['lat_clean'].isna().sum()
nan_lng_count = address_df['lng_clean'].isna().sum()
print(f"Latitudes inválidas/ausentes: {nan_lat_count}")
print(f"Longitudes inválidas/ausentes: {nan_lng_count}")

if nan_lat_count > 0 or nan_lng_count > 0:
    print("Removendo linhas com coordenadas inválidas...")
    address_df.dropna(subset=['lat_clean', 'lng_clean'], inplace=True)
    print(f"{len(address_df)} linhas restantes após remover NaNs nas coordenadas.")

# --- Pivotar para obter origem/destino por ride_id ---
# Trata possíveis RideIDs duplicados com o mesmo address_type pegando a média das coordenadas
# Se isso não for desejado, use 'first' ou 'last' no aggfunc
print("Pivotando endereços...")
try:
    address_pivoted = address_df.pivot_table(
        index='ride_id',
        columns='address_type',
        values=['lat_clean', 'lng_clean'],
        aggfunc='mean' # Usa média se houver duplicatas, 'first' pode ser alternativa
    )
    # Aplaina os nomes das colunas MultiIndex
    address_pivoted.columns = ['_'.join(map(str, col)).strip() for col in address_pivoted.columns.values]
    address_pivoted.reset_index(inplace=True)

    print(f"Tabela pivotada criada com {len(address_pivoted)} corridas únicas.")
    print(address_pivoted.head())

except Exception as e:
    print(f"Erro ao pivotar a tabela: {e}")
    # Pode ser necessário investigar duplicatas ou tipos de dados se a pivotagem falhar
    # Exemplo: Verificar se existe algum ride_id com múltiplos endereços do tipo 1 ou 2
    # duplicated_pivots = address_df[address_df.duplicated(subset=['ride_id', 'address_type'], keep=False)]
    # print("Exemplo de possíveis duplicatas que impedem a pivotagem simples:", duplicated_pivots.head())
    address_pivoted = pd.DataFrame() # Cria DataFrame vazio para evitar erro no próximo bloco

Carregando dados de endereço de: ./data/rideaddress_v1.csv
Limpando coordenadas...
Latitudes inválidas/ausentes: 0
Longitudes inválidas/ausentes: 0
Pivotando endereços...
Tabela pivotada criada com 500000 corridas únicas.
   ride_id  lat_clean_1  lat_clean_2  lng_clean_1  lng_clean_2
0  1183200   -26.329754   -26.255466   -48.840428   -48.643420
1  1183201   -27.491979   -27.437149   -48.528288   -48.398243
2  1183202   -19.849580   -19.936899   -44.019916   -43.940160
3  1183203   -23.962423   -23.837307   -46.254658   -46.132173
4  1183204   -10.919802   -10.907129   -37.077442   -37.087719


In [7]:
# --- Bloco 4: Configurar Cliente ORS e Processar Rotas (Corrigido) ---

if not address_pivoted.empty:
    print(f"Configurando cliente ORS para URL local: {ORS_BASE_URL}")
    client = None # Inicializa client como None
    ors_ready = False # Flag de prontidão

    try:
        # Tenta inicializar o cliente
        client = openrouteservice.Client(base_url=ORS_BASE_URL, timeout=60) # Timeout maior

        # Testa a conexão fazendo uma requisição simples
        print("Testando conexão com o servidor ORS local...")
        # Use coordenadas de teste válidas DENTRO DO BRASIL
        # Exemplo: São Paulo -> Campinas (Ajuste se necessário)
        test_coords = ((-46.6333, -23.5505), (-47.0608, -22.9056))
        client.directions(coordinates=test_coords, profile='driving-car')
        print("Conexão com ORS local bem-sucedida!")
        ors_ready = True # Servidor está respondendo

    except openrouteservice.exceptions.ApiError as api_err:
         # Se der erro 404 (Not Found), a conexão funcionou, mas a rota não existe
         # Isso PODE acontecer se os grafos ainda não estiverem 100% prontos ou
         # se as coordenadas de teste exatas não tiverem rota viável.
         # Consideramos como 'pronto' para tentar processar as outras.
         if api_err.status_code == 404:
             print(f"WARN: Conexão OK, mas rota de teste não encontrada (Status: {api_err.status_code}). Servidor parece estar no ar.")
             ors_ready = True
         else:
              print(f"ERRO API ORS ao testar conexão: Status {api_err.status_code}, {api_err.message}")
              print("Verifique os logs do container Docker ('docker logs ors_local').")
              ors_ready = False
              client = None # Garante que client é None se o teste falhar

    except requests.exceptions.ConnectionError as conn_err:
        print(f"ERRO: Falha de conexão ao tentar acessar {ORS_BASE_URL}.")
        print("Verifique se o container Docker 'ors_local' está rodando, se a porta 8080 está exposta e se o ORS terminou de iniciar (verifique 'docker logs ors_local').")
        print(f"Erro detalhado: {conn_err}")
        ors_ready = False
        client = None

    except Exception as e:
        print(f"ERRO inesperado ao inicializar ou testar cliente ORS: {type(e).__name__} - {e}")
        ors_ready = False
        client = None

    # --- Loop de Processamento ---
    # Execute o loop somente se o cliente foi criado e a conexão parece OK
    if client and ors_ready:
        results = []
        print(f"Iniciando processamento de rotas para {len(address_pivoted)} corridas...")

        # Itera com barra de progresso TQDM
        for index, row in tqdm(address_pivoted.iterrows(), total=len(address_pivoted), desc="Calculando Rotas"):
            ride_id = row['ride_id']
            distance = np.nan
            duration = np.nan

            # Verifica se temos ambas as coordenadas de origem (1) e destino (2)
            if pd.notna(row.get('lat_clean_1')) and pd.notna(row.get('lng_clean_1')) and \
               pd.notna(row.get('lat_clean_2')) and pd.notna(row.get('lng_clean_2')):

                # ORS espera (longitude, latitude)
                coords = ((row['lng_clean_1'], row['lat_clean_1']), (row['lng_clean_2'], row['lat_clean_2']))

                # Verifica coordenadas idênticas
                if coords[0] == coords[1]:
                    distance = 0.0
                    duration = 0.0
                else:
                    try:
                        # Chama a API de direções do ORS local
                        routes = client.directions(coordinates=coords,
                                                   profile='driving-car',
                                                   format='geojson',
                                                   preference='fastest')

                        # Extrai sumário da primeira rota encontrada
                        if routes and 'features' in routes and len(routes['features']) > 0:
                             properties = routes['features'][0].get('properties', {})
                             summary = properties.get('summary')
                             # Tenta extrair do summary global primeiro
                             if summary and isinstance(summary, dict):
                                 distance = summary.get('distance', np.nan)
                                 duration = summary.get('duration', np.nan)
                             # Fallback para segments se summary não existir ou não for dict
                             elif 'segments' in properties and len(properties['segments']) > 0:
                                 segment_summary = properties['segments'][0]
                                 distance = segment_summary.get('distance', np.nan)
                                 duration = segment_summary.get('duration', np.nan)
                             else:
                                print(f"WARN: Resposta inesperada para RideID {ride_id}. Sumário/Segmento não encontrado.")
                        else:
                            print(f"WARN: Nenhuma rota encontrada pela API para RideID {ride_id}.")


                    except openrouteservice.exceptions.ApiError as api_err:
                        if api_err.status_code == 404:
                             # 404 pode significar que os pontos estão muito longe ou não conectáveis no grafo
                             print(f"INFO: Nenhuma rota encontrada (404) para RideID {ride_id}. Coords: {coords}")
                        elif api_err.status_code == 503:
                             print(f"ERRO: Servidor ORS retornou 503 (Serviço Indisponível) para RideID {ride_id}. Pode estar sobrecarregado ou ainda inicializando grafos.")
                             time.sleep(5) # Pausa maior se o servidor estiver ocupado
                        else:
                            print(f"ERRO API ORS para RideID {ride_id}: Status {api_err.status_code}, {api_err.message}")
                    except requests.exceptions.RequestException as req_err:
                         print(f"ERRO de Rede/Conexão durante loop para RideID {ride_id}: {req_err}")
                         time.sleep(10) # Pausa maior em erro de rede
                    except Exception as e:
                        print(f"Erro inesperado processando RideID {ride_id}: {type(e).__name__} - {e}")
            else:
                 print(f"WARN: Coordenadas ausentes para RideID {ride_id}. Pulando cálculo de rota.")


            results.append({
                'ride_id': ride_id,
                'distance_m_ors_local': distance,
                'duration_s_ors_local': duration
            })

            # Pausa pequena entre requisições para não sobrecarregar CPU/IO local
            time.sleep(0.02) # Ajuste conforme necessário

        print("Processamento de rotas concluído.")

        # Cria DataFrame com os resultados
        results_df = pd.DataFrame(results)

        print("\nAmostra dos resultados calculados:")
        print(results_df.head())
        print(f"\nTotal de corridas processadas: {len(results_df)}")
        print(f"Corridas com distância calculada: {results_df['distance_m_ors_local'].notna().sum()}")
        print(f"Corridas com duração calculada: {results_df['duration_s_ors_local'].notna().sum()}")
        print(f"Corridas com NaN (erro/sem rota/pontos iguais não zero): {results_df['distance_m_ors_local'].isna().sum()}") # Conta tanto dist quanto dur

        # --- Bloco 5: Salvar Resultados ---
        print(f"\nSalvando resultados em: {OUTPUT_CSV_FILE}")
        try:
            results_df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8')
            print("Arquivo salvo com sucesso!")
        except Exception as e:
            print(f"Erro ao salvar o arquivo CSV: {e}")

    else:
         print("Não foi possível inicializar ou conectar ao servidor ORS. Nenhum cálculo de rota será feito.")

else:
     print("Tabela pivotada está vazia devido a erro anterior. Nenhum cálculo de rota será feito.")

print("\n--- Fim do Script ---")

Configurando cliente ORS para URL local: http://localhost:8080/ors
Testando conexão com o servidor ORS local...
ERRO: Falha de conexão ao tentar acessar http://localhost:8080/ors.
Verifique se o container Docker 'ors_local' está rodando, se a porta 8080 está exposta e se o ORS terminou de iniciar (verifique 'docker logs ors_local').
Erro detalhado: ('Connection aborted.', ConnectionAbortedError(10053, 'Uma conexão estabelecida foi anulada pelo software no computador host', None, 10053, None))
Não foi possível inicializar ou conectar ao servidor ORS. Nenhum cálculo de rota será feito.

--- Fim do Script ---


In [9]:
# Testa a conexão fazendo uma requisição simples
try:
    print("Testando conexão com o servidor ORS local...")
    # Use coordenadas de teste válidas dentro da área do seu PBF
    # Exemplo (substitua por coordenadas reais no Brasil se usar PBF Brasil):
    test_coords = ((-46.6333, -23.5505), (-46.6340, -23.5510)) # Exemplo SP
    client.directions(coordinates=test_coords, profile='driving-car')
    print("Conexão com ORS local bem-sucedida!")
    ors_ready = True # Flag para indicar que podemos prosseguir
except openrouteservice.exceptions.ApiError as api_err:
     # Se der erro 404 (Not Found), a conexão funcionou, mas a rota não existe (OK para teste)
     if api_err.status_code == 404:
         print(f"Conexão OK, mas rota de teste não encontrada (Status: {api_err.status_code}). Isso é normal se os grafos ainda estão construindo ou as coordenadas estão fora.")
         # Podemos continuar mesmo assim, assumindo que o serviço está de pé
         ors_ready = True
     else:
          print(f"ERRO API ORS ao testar conexão: Status {api_err.status_code}, {api_err.message}")
          print("Verifique os logs do container Docker ('docker logs ors_local').")
          ors_ready = False
except requests.exceptions.ConnectionError as conn_err:
    print(f"ERRO: Falha de conexão ao tentar acessar {ORS_BASE_URL}.")
    print("Verifique se o container Docker 'ors_local' está rodando, se a porta 8080 está exposta corretamente e se o ORS terminou de iniciar (verifique 'docker logs ors_local').")
    print(f"Erro detalhado: {conn_err}")
    ors_ready = False
except Exception as e:
    print(f"ERRO inesperado ao testar conexão com ORS local: {type(e).__name__} - {e}")
    ors_ready = False
# --- Loop de Processamento ---
# Execute o loop somente se a conexão foi bem-sucedida ou deu 404 (indicando serviço no ar)
if ors_ready:
    results = []
    print(f"Iniciando processamento de rotas para {len(address_pivoted)} corridas...")
    # ... (resto do loop como estava antes) ...
else:
    print("Não foi possível confirmar a conexão com o servidor ORS. Nenhum cálculo de rota será feito.")


print("\n--- Fim do Script ---")

Testando conexão com o servidor ORS local...
ERRO inesperado ao testar conexão com ORS local: AttributeError - 'NoneType' object has no attribute 'directions'
Não foi possível confirmar a conexão com o servidor ORS. Nenhum cálculo de rota será feito.

--- Fim do Script ---
